In [2]:
import argparse
from enum import Enum
import io
import os
from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import math

In [9]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'
client = vision.ImageAnnotatorClient()

bounds = []

with io.open(os.path.join(r'D:\SiH-Frame-Project\MyVisionDemo\Images','abc.jpeg'), 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

response = client.document_text_detection(image=image)
document = response.full_text_annotation

In [10]:
frame = pd.DataFrame(columns=['x1','y1','x2','y2','x3','y3','x4','y4','area','text']) 

In [11]:
x1=0
x2=0
x3=0
x4=0
y1=0
y2=0
y3=0
y4=0
text=''
for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                x1=paragraph.words[0].symbols[0].bounding_box.vertices[0].x
                x2=paragraph.words[0].symbols[0].bounding_box.vertices[1].x
                x3=paragraph.words[0].symbols[0].bounding_box.vertices[2].x
                x4=paragraph.words[0].symbols[0].bounding_box.vertices[3].x
                y1=paragraph.words[0].symbols[0].bounding_box.vertices[0].y
                y2=paragraph.words[0].symbols[0].bounding_box.vertices[1].y
                y3=paragraph.words[0].symbols[0].bounding_box.vertices[2].y
                y4=paragraph.words[0].symbols[0].bounding_box.vertices[3].y
                area=math.sqrt((x2-x1)**2+(y2-y1)**2)*math.sqrt((x3-x2)**2+(y3-y2)**2)
                for word in paragraph.words:
                    for symbol in word.symbols:
                        text+=text.join(symbol.text)
                    text+=text.join(' ')
                frame=frame.append(dict(x1=x1,
                                  x2=x2,
                                  x3=x3,
                                  x4=x4,
                                  y1=y1,
                                  y2=y2,
                                  y3=y3,
                                  y4=y4,
                                  area=area,
                                  text = text),ignore_index=True)
                text=''
               

In [12]:
frame

,x1,y1,x2,y2,x3,y3,x4,y4,area,text
0,565,166,565,171,551,170,551,165,70.178344,Aceclofenac IP
1,553,331,553,334,536,332,536,329,51.351728,100g Colours : Sunset Yellow ECF &
2,536,172,536,175,522,174,522,171,42.107007,tanium Dioxide JP Dosage : As directed by the ...
3,447,257,447,262,435,261,435,256,60.207973,al l the prescription
4,548,438,548,440,537,439,537,437,22.090722,LOU
5,536,569,534,597,523,596,525,568,310.058059,"ILLES PORN 28A - 50 Sector LE , SICH Handyre 2..."
6,498,432,497,440,483,439,484,431,113.159180,copini Toll Free no . : 1606 . 2008 Spee dsm @...
7,466,454,465,461,448,459,449,452,121.037184,"Marketed by : LUPIN LTD . 159 , C . S . T . Ro..."
8,417,629,421,629,420,643,416,643,56.142675,20111517
9,377,660,385,660,384,677,376,677,136.235091,Mig . Lic . No . : 31 / 0A / 2013


In [13]:
list(frame[frame['area']==max(frame['area'])]['text'])

['Aceclofenac Tablets IP . 108 Acemiz . ']